## Generacion de CSV (Opcional)

Primero ejecutamos el codigo para generar el archivo csv desde el cual se creara el dataframe, al modificar el valor de "pages" se pueden obtener mas o menos datos segun sea necesario. Es importante recalcar que esto solo funcionara si se tiene una RAWG_API_KEY para acceder a la API.

In [ ]:
import sys
import os

sys.path.append(os.path.abspath(".."))

import app

nombreArchivo = "steam_sample.csv"

dfe = app.extract_data(pages=10)
dfe.to_csv("../data/"+nombreArchivo, index=False)
print("Datos guardados en "+nombreArchivo)

Datos guardados en steam_sample2.csv


Primeramente haremos los imports necesarios y cargaremos el dataframe, mientras que asignamos lo que catalogaremos como "exito", lo cual serian puntuaciones "buenas" por RawG y Metacritic, y que hayan mas usuarios interesados que reseñas en RawG.

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

df = pd.read_csv("../data/steam_sample.csv")

df["success"] = (df["rating"] >= 4) & (df["metacritic"] >= 75) & (df["ratings_count"] < df["usuarios_interesados"])

Aqui vemos como quedaron los resultados de la clasificacion de juegos "exitosos"

In [2]:
print(df["success"].value_counts())

success
False    234
True     166
Name: count, dtype: int64


Ahora entrenamos el modelo para que pueda predecir el "exito" de un juego basado en los generos del juego.


## División del conjunto de datos

Se dividió el conjunto de datos en tres subconjuntos: entrenamiento (60%), validación (20%) y prueba (20%). 
Se utilizó la técnica de estratificación para asegurar que la proporción de clases se mantuviera constante en cada subconjunto, 
lo que es especialmente importante cuando se trabaja con clases desbalanceadas.


In [ ]:
mlb_genres = MultiLabelBinarizer()
genres_encoded = mlb_genres.fit_transform(df["generos"].apply(eval))  # listas

X = pd.DataFrame(genres_encoded, columns=mlb_genres.classes_)
X["ratings_count"] = df["ratings_count"]
X["metacritic"] = df["metacritic"].fillna(0)
y = df["success"]

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=42)

,Modelo,Accuracy,Precision,Recall,F1 Score
0,Random Forest,0.8375,0.857143,0.789474,0.821918
2,Logistic Regression,0.8125,0.870968,0.710526,0.782609
1,SVM,0.6750,0.730769,0.500000,0.593750


## Evaluacion de algoritmos de clasificacion

En este segmento se probaran los algoritmos Random Forest, SVC y Logistic Regression, para analizar sus resultados.

In [ ]:
models = {
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(),
    "Logistic Regression": LogisticRegression(max_iter=2000)
}

results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    results.append({
        "Modelo": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred)
    })

pd.DataFrame(results).sort_values(by="F1 Score", ascending=False)

Aqui hay un ejemplo para ver que predice el modelo con respecto al "exito" que tendra un juego basado en sus generos y la probabilidad de que se cumpla la prediccion.

In [4]:
def predecir_exito(generos, ratings_count=1000, metacritic=80):
    input_data = pd.DataFrame([0]*len(mlb_genres.classes_), index=mlb_genres.classes_).T
    for genre in generos:
        if genre in input_data.columns:
            input_data[genre] = 1
    input_data["ratings_count"] = ratings_count
    input_data["metacritic"] = metacritic

    pred = model.predict(input_data)[0]
    prob = model.predict_proba(input_data)[0][1]
    return bool(pred), prob

exito, probabilidad = predecir_exito(["Family"], ratings_count=500, metacritic=85)
print("¿Será exitoso?", exito)
print("Probabilidad:", probabilidad)

¿Será exitoso? True
Probabilidad: 0.5408173516412131


## Selección y ajuste de hiperparámetros

Se utilizó la técnica de Grid Search con validación cruzada (5-fold) para encontrar los mejores hiperparámetros del modelo Random Forest.
Se evaluaron `n_estimators` y `max_depth` por su impacto directo en el sesgo y varianza del modelo.

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20]
}

clf = RandomForestClassifier(class_weight='balanced', random_state=42)
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Mejores hiperparámetros:", grid_search.best_params_)
best_model = grid_search.best_estimator_

Mejores hiperparámetros: {'max_depth': 10, 'n_estimators': 100}


## Manejo de clases desbalanceadas

Se detectó un desbalance en las clases, por lo tanto, se empleó la estrategia `class_weight='balanced'` en los modelos 
para ajustar los pesos inversamente proporcionales a la frecuencia de las clases.

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

svm_model = SVC(class_weight='balanced', random_state=42)
log_model = LogisticRegression(class_weight='balanced', random_state=42)

## Métricas de evaluación

Se evaluaron los modelos utilizando precisión, recall y F1-score, ya que este último es una medida armónica que balancea precisión y recall, 
lo cual es fundamental en contextos con clases desbalanceadas.

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_model(model, X, y, label="Validación"):
    y_pred = model.predict(X)
    print(f"Evaluación en {label}:")
    print("Precisión:", precision_score(y, y_pred))
    print("Recall:", recall_score(y, y_pred))
    print("F1-score:", f1_score(y, y_pred))

evaluate_model(best_model, X_val, y_val, "Validación")
evaluate_model(best_model, X_test, y_test, "Prueba")

## Interpretación de resultados

Se compararon los resultados de distintos modelos usando tablas para identificar sus fortalezas y debilidades. 
Esto permite observar cómo se desempeñan en métricas clave y tomar decisiones informadas.

In [ ]:
results = []

models = {
    "Random Forest": best_model,
    "SVM": svm_model.fit(X_train, y_train),
    "Logistic Regression": log_model.fit(X_train, y_train)
}

for name, model in models.items():
    y_pred = model.predict(X_val)
    results.append({
        "Modelo": name,
        "Precisión": precision_score(y_val, y_pred),
        "Recall": recall_score(y_val, y_pred),
        "F1-score": f1_score(y_val, y_pred)
    })

import pandas as pd
results_df = pd.DataFrame(results)
print(results_df)

## Selección del modelo final

Se seleccionó el modelo con el mayor F1-score como el más adecuado para el problema. Este criterio considera tanto 
la precisión como el recall, lo que es ideal cuando las clases están desbalanceadas.

In [ ]:
best_row = results_df.loc[results_df["F1-score"].idxmax()]
print("Modelo seleccionado:", best_row["Modelo"])

## Reproducibilidad del experimento

Para garantizar la reproducibilidad del experimento, se fijaron las semillas aleatorias y se recomienda mantener un entorno 
consistente mediante control de versiones y especificación de dependencias.

In [ ]:
import numpy as np
import random

np.random.seed(42)
random.seed(42)

## Uso de GPT

A lo largo del desarrollo se utilizo ChatGPT para ayudar a consolidar la informacion dentro de las casillas de texto, asi como en la generacion de codigo, resultando al final en un archivo mezclado con las casillas hechas por nosotros y ChatGPT.